# Stochastic Analysis of a simple Repressor-Recombinase oscillator: RR Design

In this Jupyterbook, we implement the Gilleaspy algorithm of a recombinase-based oscillator at a single copy 

In [20]:
# Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
  
    from google.colab import drive
    drive.mount('/content/drive')
    dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/sRNARecombinase/"
else:
    dir = "/Users/christian/My Drive/Research/2022/RecombinaseOscillator/sRNARecombinase/"

# ------

# This check the number of processors in the computer
#% cat /proc/cpuinfo | grep processor | wc -l

# % pip install biocircuits bokeh-catplot watermark blackcellmagic
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import numba
import math

from scipy.signal import argrelextrema
import scipy.fftpack
import os.path

import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.io import export_svgs


bokeh.io.output_notebook()

Loading BokehJS ...

We start by writing a function to create two matrices of differense size by placing them in a diagonal [A, 0 ; 0, B]

In [21]:
def JoinMatrices(A,B):
    n1, m1 = A.shape
    n2, m2 = B.shape
    ZA = np.zeros((n1,m2))
    ZB = np.zeros((n2,m1))
    tmp1 = np.hstack((A,ZA))
    tmp2 = np.hstack((ZB,B)) 
    return np.vstack((tmp1,tmp2))

Next, we write down the stoichiometry matrices for all the reactions. We beging with the stoichiometry matrix for a single switch $S_1$, and then use the function "Join Matrixes" to create the stoichiometry matrix for two switches $S_2$. $S_2$ represent the promoter pointing to the right and the left. Therefore, we also add the transition between these two states (the last part of the code).

In [22]:
# Stoichiometry  matrix of a single switch
# R00, R10, R01, R11
S1 = np.array(
    [
     [-1, 1, 0, 0],  # R00 -> R10
     [ 1, -1, 0, 0], # R10 -> R00 
     [ 0, -1, 0, 1], # R10 -> R11
     [ 0, 1, 0, -1], # R11 -> R10

     [-1, 0, 1, 0],  # R00 -> R01
     [ 1, 0, -1, 0], # R01 -> R00 
     [ 0, 0, -1, 1], # R01 -> R11
     [ 0, 0, 1, -1], # R11 -> R01 

     [ 0, 0,  0, -1], # R11 -> L00
    ],
    dtype=int,
)
# Stoichiometry  matrix of two switches
S2 = JoinMatrices(S1,S1)
S2[8,4] = 1
S2[17,0] = 1
#print(S2)

Now, we describe the stoichiometry matrix $S_3$. it describe the trsnscription of mRNA recombinase, translation of recombianse, and the decays of mRNA and protein. It also incorporates the heterodimer formation between both recombinases $S_4$. In addition, it updates the number of recombinases require for each flip. Finally, we define the transcription, translation, and decays of mRNA and protein of repressprs $S_5$. Combining all of them results in the stoichiometry matrix of the whole system.

In [23]:
# Stoichiometry  matrix of transcription and translation
# m1, x1, y1, m2, x2, y2, c
S3 = np.array(
    [
     [ 1, 0, 0, 0, 0, 0, 0], # m1 production (TX)
     [-1, 0, 0, 0, 0, 0, 0], # m1 decay
     [ 0, 1, 0, 0, 0, 0, 0], # x1 production (TL)
     [ 0,-1, 0, 0, 0, 0, 0], # x1 decay
     [ 0,-2, 1, 0, 0, 0, 0], # y1 production (dimerization)
     [ 0, 2,-1, 0, 0, 0, 0], # y1 dissociation
     [ 0, 0,-1, 0, 0, 0, 0], # y1 decay

     [ 0, 0, 0, 1, 0, 0, 0], # m2 production (TX)
     [ 0, 0, 0,-1, 0, 0, 0], # m2 decay
     [ 0, 0, 0, 0, 1, 0, 0], # x2 production (TL)
     [ 0, 0, 0, 0,-1, 0, 0], # x2 decay
     [ 0, 0, 0, 0,-2, 1, 0], # y2 production (dimerization)
     [ 0, 0, 0, 0, 2,-1, 0], # y2 dissociation
     [ 0, 0, 0, 0, 0,-1, 0], # y2 decay

     [ 0,-1, 0, 0,-1, 0, 1], # c production (heterodimer formation)
     [ 0, 1, 0, 0, 1, 0,-1], # c heterodimer dissociation
     [ 0, 0, 0, 0, 0, 0,-1], # c decay
    ],
    dtype=int,
)
# Stoichiometry  matrix of two switches
S4 = JoinMatrices(S2,S3)
# Updating the consumption of recombinase in the switches
S0 = np.array(
    [
     [-1, 1, -1, 1,-1, 1, -1, 1, 2],  # R00 -> R10
    ],
    dtype=int,
)
S4[0:9,10] = S0
S4[9:18,13] = S0

S5 = np.array(
    [
     [ 1, 0], # sRNA production (TX)
     [-1, 0], # sRNA decay
     [-1, 0], # sRNA and mRNA (R1) sequestration
        
     [ 0, 1], # sRNA production (TX)
     [ 0,-1], # sRNA decay
     [ 0,-1], # sRNA and mRNA (R2) sequestration
    ],
    dtype=int,
)

simple_update = JoinMatrices(S4,S5)
simple_update[37,8] = -1
simple_update[40,11] = -1
#print(simple_update)
print(S4.shape)
print(simple_update.shape)

(35, 15)
(41, 17)


In this part of the code, we write down the propensity of all reactions. In addition, we convert the rates from concentration to molecules throught the parameter Omega. 

In [24]:
def simple_propensity(propensities, x, t, th, rh, ph, d, g, gd, a0, d0, r, b, psi, Omega, n):
    #x, y, c = population
    # Switch 1 (Promoter pointing to the right R)
    propensities[0] = a0 * x[0] * x[10]  / Omega # R00 -> R10
    propensities[1] = d0 * x[1] # R10 -> R00
    propensities[2] = a0 * x[1] * x[10]  / Omega # R10 -> R11
    propensities[3] = d0 * x[3] # R11 -> R10
    propensities[4] = a0 * x[0] * x[10]  / Omega # R00 -> R01
    propensities[5] = d0 * x[2] # R01 -> R00
    propensities[6] = a0 * x[2] * x[10]  / Omega # R01 -> R11
    propensities[7] = d0 * x[3] # R11 -> R01
    propensities[8] = r * x[3] # R11 -> L00

    # Switch 2 (Promoter pointing to the left L)
    propensities[ 9] = a0 * x[4] * x[13]  / Omega # L00 -> L10
    propensities[10] = d0 * x[5] # L10 -> L00
    propensities[11] = a0 * x[5] * x[13]  / Omega # R10 -> R11
    propensities[12] = d0 * x[7] # R11 -> R10
    propensities[13] = a0 * x[4] * x[13]  / Omega # R00 -> R01
    propensities[14] = d0 * x[6] # R01 -> R00
    propensities[15] = a0 * x[6] * x[13]  / Omega # R01 -> R11
    propensities[16] = d0 * x[7] # R11 -> R01
    propensities[17] = r * x[7] # R11 -> L00

    # TX-TL of recombinase 1
    propensities[18] = th * x[0]# R00 -> R00 + M1
    propensities[19] = ph * x[8] # M1 -> 0
    propensities[20] = rh * x[8] # M1 -> M1 + X1
    propensities[21] = d * x[9] # X1 -> 0
    propensities[22] = g * x[9] * (x[9]-1)  / 2 / Omega # X1 + X1 -> Y1
    propensities[23] = gd * x[10] # Y1 -> X1 + X1
    propensities[24] = d * x[10] # Y1 -> 0

    # TX-TL of recombinase 2
    propensities[25] = th * x[4]# L00 -> L00 + M2
    propensities[26] = ph * x[11] # M2 -> 0
    propensities[27] = rh * x[11] # M2 -> M2 + X2
    propensities[28] = d * x[12] # X2 -> 0
    propensities[29] = g * x[12] * (x[12]-1)  / 2 / Omega # X2 + X2 -> Y2
    propensities[30] = gd * x[13] # Y2 -> X2 + X2
    propensities[31] = d * x[13] # Y2 -> 0

    # Heterodimmer reaction
    propensities[32] = g * x[9] * x[12] / Omega # L00 -> L00 + M2
    propensities[33] = gd * x[14] # M2 -> 0
    propensities[34] = d * x[14] # M2 -> M2 + X2 
    
    # sRNA production and sequestration of mRNA R1
    propensities[35] = b  # mRNA Y1 repressor 1 (TX)
    propensities[36] = ph * x[15] # mRNA Y1 -> 0
    propensities[37] = psi * x[15] * x[8] / Omega # mRNA -> mRNA + Y1 (TL) 
    
    # sRNA production and sequestration of mRNA R1
    propensities[38] = b  # mRNA Y2 repressor 2 (TX)
    propensities[39] = ph * x[16] # mRNA -> 0
    propensities[40] = psi * x[16] * x[11] / Omega # mRNA -> mRNA + Y1 (TL)  


Now, we are ready to realize the stochastic simulations of the whole system.

In [45]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = 0.3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching rate
a0 = 2.1*36
d0 = 0.1*a0

b = 0.3*60/n
psi = 2.1*36

Omega = 600 # 600
state = (10,13) # 10/13(R1/R2) Y1/Y2 (16/20) C1/C2 (17/21)

args = (th, rh, ph, d, g, gd, a0, d0, r, b, psi, Omega, n)
time_points = np.linspace(0, 40, 401)
population_0 = np.zeros(17, dtype=int)
population_0[0] = n

samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=1, args=args, n_threads=1,progress_bar=False)

# Set up plots
fig_size = [600, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points[-1])
y_range = (0, 400)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"


# Plot trajectories of recombinase 1 and 2
y1 = samples[0,:,state[0]] # samples[:,:,10][0]
plots[0].step(time_points, y1, line_width=2, alpha= 1, line_join="bevel",color="grey")

y2 = samples[0,:,state[1]]
plots[0].step(time_points, y2, line_width=2, alpha= 1, line_join="bevel",color="orange")
# Link axes

bokeh.io.show(bokeh.layouts.row(plots))

In [19]:
# Set up plots
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [266, 115] # visualize (600/200)
x_range = (0, 40)
y_range = (0, 100)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

# Plot trajectories of recombinase 1 and 2
sR = samples[0,:,4:8].sum(axis=1)

plots[0].varea(time_points, sR*0, sR*y_range[1], fill_color="#e8f6f7")

# Plot trajectories of recombinase 1 and 2
y1 = samples[0,:,state[0]] # samples[:,:,10][0]
plots[0].step(time_points, y1, line_width=2, alpha= 0.5, line_join="bevel",color="#999999")

y2 = samples[0,:,state[1]]
plots[0].step(time_points, y2, line_width=2, alpha= 1, line_join="bevel",color="#18a3ac")
# Link axes

bokeh.io.show(bokeh.layouts.row(plots))
plots[0].output_backend = "svg"
export_svgs(plots,filename="fig/SingleTrajectory.svg")

['fig/SingleTrajectory.svg']

We will evaluate the period of oscillations from the stochastic ttrajaectories by computing the auto-correlation function.

In [7]:
# Computing the auto-correlation

def correlate(x, y, dt):
    # Compute correlation with max correlation being unity
    corr = np.correlate(x, y, "full")
    corr /= corr.max()

    idx = np.where(corr==1.)[0][0]
    corr = corr[idx:]
    lags = np.arange(0, len(corr))*dt
    return lags, corr


In [ ]:
fig_size = [400, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range, 
                          #y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

dt = time_points[1] - time_points[0]
y1 = samples[0,:,10] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)
max_id = argrelextrema(auto_corr, np.greater)

plots[0].step(time_points, y1, line_width=2, alpha= 1, line_join="bevel",color="orange")
#plots[0].line(lags, auto_corr*130, line_width=2, alpha= 1, line_join="bevel",color="black")

plots[1].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="black")
plots[1].circle(lags[max_id], auto_corr[max_id], line_width=2, alpha= 1, line_join="bevel",color="red")


y1 = samples[0,:,13] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)
max_id = argrelextrema(auto_corr, np.greater)

plots[2].step(time_points, y1, line_width=2, alpha= 1, line_join="bevel",color="orange")
#plots[2].line(lags, auto_corr*130, line_width=2, alpha= 1, line_join="bevel",color="black")

plots[3].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="black")
plots[3].circle(lags[max_id], auto_corr[max_id], line_width=2, alpha= 1, line_join="bevel",color="red")

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))

In [ ]:
def FFT(t1,x1,fr):
    n = len(x1)
    if n % 2:
        x = x1[:-2]
        t = t1[:-2]
        n = len(x)
    else:
        x = x1[:-1]
        t = t1[:-1]

    dt = (t[1]-t[0])
    tf = scipy.fftpack.fftfreq(x.size, d=dt)
    xf = scipy.fftpack.fft(x)
    xf = np.abs(xf)**2
    idx = np.where((tf >= fr[0]) & (tf <=fr[1]))
    return tf[idx],xf[idx]

In [ ]:
fig_size = [400, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

dt = time_points[1] - time_points[0]

y1 = samples[0,:,10] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)
f,Mf = FFT(lags, auto_corr, (0, 2))

plots[0].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="orange")
plots[1].line(f, Mf, line_width=2, alpha= 1, line_join="bevel",color="orange")

y1 = samples[0,:,13] # samples[:,:,10][0]
y1s = y1 - y1.mean()
lags, auto_corr = correlate(y1s, y1s, dt)
f,Mf = FFT(lags, auto_corr, (0, 2))

plots[0].line(lags, auto_corr, line_width=2, alpha= 1, line_join="bevel",color="black")
plots[1].line(f, Mf, line_width=2, alpha= 1, line_join="bevel",color="black")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = .1*600/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching rate
a0 = 2.1*36
d0 = 0.2*a0

Omega = 600 # 600
size = 250
#dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/"
f_name = dir+"Nodel_1_Data_"+str(size)+".npy"

args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)
time_points = np.linspace(0, 40, 401)
population_0 = np.zeros(15, dtype=int)
population_0[0] = n

if os.path.exists(f_name):
    samples = np.load(f_name)
else:
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=size, args=args, n_threads=2,progress_bar=False)
    #np.savetxt(f_name, samples, fmt='%d')
    np.save(f_name, samples)


In [ ]:
# This function finds the time of the peak
def FindPicks(t,samples):
    ys = samples - samples.mean()
    lags, auto_corr = correlate(ys, ys, t[1]-t[0])
    max_id = argrelextrema(auto_corr, np.greater)
    return t[max_id[0][0:4]]

In [ ]:
t_peaks = FindPicks(time_points,samples[0,:,10])
for i in range(1,samples.shape[0]):
    t_peaks = np.vstack((t_peaks,FindPicks(time_points,samples[i,:,10])))

#print(tmp)

In [ ]:
fig_size = [300, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

x = np.linspace(0,20,41)

for j in range(3):
    hist, bin_edges = np.histogram(t_peaks[:,j], x, density=True)
    plots[j].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],fill_color="grey", line_color="grey", fill_alpha=0.2,line_alpha=0.)
    plots[j].step(bin_edges[:-1],hist, line_width=2, mode="after", color = "grey")

bokeh.io.show(bokeh.layouts.row(plots))

In [8]:
# This function finds the Delta T between peak times
def FindDeltaT(t,samples):
    ys = samples - samples.mean()
    lags, auto_corr = correlate(ys, ys, t[1]-t[0])
    max_id = argrelextrema(auto_corr, np.greater)
    return t[max_id[0][1:5]]-t[max_id[0][0]]

In [ ]:
Delta_T = FindDeltaT(time_points,samples[0,:,10])
for i in range(1,samples.shape[0]):
    Delta_T = np.vstack((Delta_T,FindDeltaT(time_points,samples[i,:,10])))

In [ ]:
fig_size = [300, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

x = np.linspace(0,20,41)

for j in range(3):
    hist, bin_edges = np.histogram(Delta_T[:,j], x, density=True)
    plots[j].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],fill_color="grey", line_color="grey", fill_alpha=0.2,line_alpha=0.)
    plots[j].step(bin_edges[:-1],hist, line_width=2, mode="after", color = "grey")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
# Computing the peak vs the variance

fig_size = [300, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points[-1])
y_range = (0, 200)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

vx = range(1,5)
vy = np.var(Delta_T, axis=0)

plots[0].circle(vx,vy, line_width=2)

L_eq = np.polyfit(vx,vy,1)
plots[0].line(vx, np.poly1d(L_eq)(vx), line_width=2, alpha= 0.5, line_join="bevel",color="grey")

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
L_eq

In [ ]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = 0.3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching rate
a0 = 2.1*36/2
d0 = 0.1*a0

b = 0.3*60/n
psi = 2.1*36

Omega = 600 # 600
size = 2
state = 10 # 10 (recombinase 1) or 13 (recombinase 2)
args = (th, rh, ph, d, g, gd, a0, d0, r, b, psi, Omega, n)

time_points = np.linspace(0, 40, 401) # 40 by 401
population_0 = np.zeros(15, dtype=int)
population_0[0] = n

#param = (0,1,2)
param = range(0,8)
scale = (1./4, 1./3, 1./2, 1., 2.,3., 4.)
vx = range(1,5)

out = np.array((0., 1./4, 1./3, 1./2, 1., 2., 3., 4.))

#dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/"
f_name2 = dir+"Model_1_AllSensitivity_"+str(size)+".npy"

if os.path.exists(f_name2):
    out = np.load(f_name2)
    idx = param.index(out[-1,0])+1
else:
    idx = 0

if idx<len(param):
    for par in param[idx:]:
        print(par)
        tmp = np.array((par))
        args_i = np.array((th, rh, ph, d, g, gd, a0, d0, r, b, psi, Omega, n))
        for fac in scale:
            # Update model para
            args_i[par] = args[par]*fac
            samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=size, args=tuple(args_i), n_threads=2,progress_bar=False)

            # Stack T1, T2, T3, T4
            Delta_T = FindDeltaT(time_points,samples[0,:,state])
            for i in range(1,samples.shape[0]):
                Delta_T = np.vstack((Delta_T,FindDeltaT(time_points,samples[i,:,state])))

            # Compute variance and find slope (incoherent metric)
            vy = np.var(Delta_T, axis=0)
            L_eq = np.polyfit(vx,vy,1)
            tmp = np.append(tmp,L_eq[0])
    
        out = np.vstack((out,tmp))
        np.save(f_name2, out)

print(out)

[[ 0.00000000e+00  2.50000000e-01  3.33333333e-01  5.00000000e-01
   1.00000000e+00  2.00000000e+00  3.00000000e+00  4.00000000e+00]
 [ 0.00000000e+00 -1.26000000e-01  1.06850000e+00  2.88062500e-01
   1.06062500e-01  2.14500000e-01  1.11125000e+00  7.46250000e-02]
 [ 1.00000000e+00  2.36312500e+00  1.52500000e-02  3.18950000e+00
  -4.59375000e-02 -7.28125000e-02 -1.51812500e-01  3.11562500e-01]
 [ 2.00000000e+00  1.38437500e-01  1.23687500e-01  1.37943750e+00
   2.58750000e-01  4.00625000e-02  1.06100000e+00  6.19750000e-01]
 [ 3.00000000e+00  4.81312500e-01  8.29437500e-01  8.20312500e-01
  -6.74375000e-02 -1.79375000e-02  7.50062500e-01  1.26312500e-01]
 [ 4.00000000e+00  4.85000000e-01 -1.41062500e-01  3.98750000e-01
   2.58993750e+00  5.10312500e-01  5.95937500e-01  1.67687500e-01]
 [ 5.00000000e+00  1.49843750e+00  4.13150000e+00  1.60750000e-01
   4.79875000e-01  9.69375000e-01  1.29456250e+00  9.17500000e-01]
 [ 6.00000000e+00  2.81250000e-03  1.17250000e-01  1.10187500e-01
   

In [2]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = 0.3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching rate
a0 = 2.1*36/2
d0 = 0.1*a0

b = 0.3*60/n
psi = 2.1*36

Omega = 600 # 600
size = 500
state = 13 # 10 (recombinase 1) or 13 (recombinase 2)
args = (th, rh, ph, d, g, gd, a0, d0, r, b, psi, Omega, n)

time_points = np.linspace(0, 40, 401) # 40 by 401
population_0 = np.zeros(17, dtype=int)
population_0[0] = n

param = (1,3,4,6,8,9)
#param = range(0,8)
scale = (1./4, 1./3, 1./2, 1., 2.,3., 4.)
vx = range(1,5)

out = np.array((0., 1./4, 1./3, 1./2, 1., 2., 3., 4.))

#dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/"
f_name2 = dir+"Model_2_AllSensitivity_"+str(size)+".npy"

if os.path.exists(f_name2):
    out = np.load(f_name2)
    idx = param.index(out[-1,0])+1
else:
    idx = 0

if idx<len(param):
    for par in param[idx:]:
        print(par)
        tmp = np.array((par))
        args_i = np.array((th, rh, ph, d, g, gd, a0, d0, r, b, psi, Omega, n))
        for fac in scale:
            # Update model para
            args_i[par] = args[par]*fac
            ii = 0
            Delta_T = np.array((0.,0.,0.,0.))
            while ii<size+1:
                samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=1, args=tuple(args_i), n_threads=1,progress_bar=False)
                try:
                  tmp_0 = FindDeltaT(time_points,samples[0,:,state])
                  if len(tmp_0) == 4:
                      Delta_T = np.vstack((Delta_T,tmp_0[0:4]))
                      ii = ii + 1
                except: tmp_0 = 0

            # Compute variance and find slope (incoherent metric)
            vy = np.var(Delta_T[1:,:], axis=0)
            L_eq = np.polyfit(vx,vy,1)
            tmp = np.append(tmp,L_eq[0])
    
        out = np.vstack((out,tmp))
        np.save(f_name2, out)

print(out)

[[ 0.          0.25        0.33333333  0.5         1.          2.
   3.          4.        ]
 [ 1.         14.4767014  11.85378802  7.60441036  3.64409961  1.63767488
   1.25120237  0.96014259]
 [ 3.          5.72558935  4.82662409  4.36118443  3.11681635  4.5846496
  11.78460984 13.59759152]
 [ 4.          5.08539341  4.18833793  3.7701348   3.39811145  3.1081451
   3.40261171  3.50222716]
 [ 6.         11.33045389  8.48343638  7.34326921  3.25609339  1.74096699
   1.43823087  1.37624998]
 [ 8.          8.52035522  7.2321941   5.13109403  3.43609798  2.7207024
   2.40394232  2.14594765]
 [ 9.          3.62219413  3.56097747  3.15463571  3.50229582  3.64095008
   4.20195797  4.33297243]]


In [4]:
# Set up plots
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [144, 115] # for paper size (260/115), half size (144,115), visualize (600/200)

x_range = (0, 4.2)
y_range = (0, 20)

plots = []
for i in range(out.shape[0]-1):
    #print(i)
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

color="#18a3ac"    
for i in range(out.shape[0]-1):
    plots[i].line(out[0,1:], out[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color=color)
    plots[i].circle(out[0,1:], out[i+1,1:], line_width=2, alpha= 1, line_join="bevel",color=color)
    #plots[i].xaxis.ticker = list(np.linspace(0,4,5))
    #plots[i].x_range = Range1d(0, 4)
    plots[i].output_backend = "svg"
    export_svgs(plots[i],filename="fig/Sensitivity_Param_" + str(out[i+1,0]) + ".svg")

bokeh.io.show(bokeh.layouts.row(plots))